In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import json

### Análisis y Consideraciones
1. **Características Enriquecidas en el Train**: 
   - Nuevas columnas como `session_interactions`, `discount_category`, `similar_products`, `popularity`, `cluster`, etc., ofrecen una riqueza de información sobre el comportamiento de los usuarios y los productos.
   - Sin embargo, muchas de estas características no están presentes en el dataset de prueba (`test_data`). Por ejemplo, `add_to_cart`, `session_interactions`, `discount_category`, y `similar_products` no aparecen en `test`.

2. **Dataset de Prueba**:
   - Contiene columnas básicas (`session_id`, `date`, `timestamp_local`, `user_id`, `country`, `partnumber`, `device_type`, `pagetype`).
   - Por lo tanto, el modelo debe ser diseñado para funcionar únicamente con las características que también están disponibles en el dataset de prueba.

3. **Aprovechar el Dataset Enriquecido**:
   - Utilizar las características enriquecidas para crear interacciones más complejas en el entrenamiento.
   - Generar características derivadas basadas en patrones que puedan ser representadas con las columnas del dataset de prueba.

4. **Objetivo**:
   - Diseñar un modelo (como LambdaMART) que utilice características compartidas entre `train` y `test`.
   - Introducir transformaciones o modelos adicionales que puedan inferir las características ausentes en el dataset de prueba (por ejemplo, estimar la `popularidad` de productos en base a patrones previos).



---


### Estrategia Propuesta
#### 1. **Selección de Características**
   - Seleccionar solo las características comunes entre `train` y `test`:
     - `session_id`, `date`, `timestamp_local`, `user_id`, `country`, `partnumber`, `device_type`, `pagetype`.
   - Generar características adicionales que puedan calcularse tanto en `train` como en `test`, como:
     - **Interacciones por sesión** (`session_length`).
     - **Popularidad relativa del producto** (`product_popularity`).
     - **Hora y día de interacción** (`hour`, `day_of_week`).

In [2]:
def preprocess_and_generate_shared_features(df, mode="train"):
    print(f"\n--- Preprocesando y generando características compartidas ({mode}) ---")

    # Convertir datetime a numérico (segundos desde el epoch)
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"]).astype(int) / 10**9
    if "timestamp_local" in df.columns:
        df["timestamp_local"] = pd.to_datetime(df["timestamp_local"]).astype(int) / 10**9

    # Generar características compartidas
    if "partnumber" in df.columns:
        df["product_popularity"] = df.groupby("partnumber")["session_id"].transform("nunique")
    if "session_id" in df.columns:
        df["session_length"] = df.groupby("session_id")["partnumber"].transform("count")

    # Codificar `discount_category` como numérico
    if "discount_category" in df.columns:
        df["discount_category"] = df["discount_category"].astype("category").cat.codes

    # Procesar `similar_products` como longitud de lista
    if "similar_products" in df.columns:
        df["similar_products_count"] = df["similar_products"].apply(lambda x: len(x) if isinstance(x, list) else 0)
        df.drop(columns=["similar_products"], inplace=True)

    # Manejar columnas no compatibles en modo `test`
    if mode == "test":
        for col in df.columns:
            if df[col].dtype not in [np.int32, np.int64, np.float32, np.float64, np.bool_]:
                print(f"Eliminando columna no compatible: {col}")
                df.drop(columns=[col], inplace=True)

    return df

In [3]:
def generate_global_stats(train_df):
    print("\n--- Generando estadísticas globales del entrenamiento ---")
    stats = {}
    stats['product_popularity'] = train_df.groupby('partnumber')['session_id'].nunique()
    stats['discount_category'] = train_df['discount_category'].value_counts(normalize=True).to_dict()
    stats['cluster'] = train_df['cluster'].value_counts(normalize=True).to_dict()

    # Asignar valores por defecto para test
    stats['discount_category']['default'] = max(stats['discount_category'], key=stats['discount_category'].get)
    stats['cluster']['default'] = max(stats['cluster'], key=stats['cluster'].get)
    return stats


In [4]:
def preprocess_test_with_enrichment(test_df, global_stats, train_features):
    print("\n--- Preprocesando conjunto de prueba con enriquecimiento ---")

    if 'session_id' not in test_df.columns:
        raise KeyError("El conjunto de prueba no contiene 'session_id' al inicio.")

    # Convertir columnas datetime a numérico
    test_df['date'] = pd.to_datetime(test_df['date']).astype(int) / 10**9
    test_df['timestamp_local'] = pd.to_datetime(test_df['timestamp_local']).astype(int) / 10**9
    test_df['hour'] = pd.to_datetime(test_df['timestamp_local'], unit='s').dt.hour
    test_df['day_of_week'] = pd.to_datetime(test_df['timestamp_local'], unit='s').dt.dayofweek

    # Generar características derivadas
    test_df['session_length'] = test_df.groupby('session_id')['partnumber'].transform('count')
    test_df['product_popularity'] = global_stats['product_popularity'].reindex(test_df['partnumber']).fillna(0).astype(float).values

    # Manejar columna `discount_category`
    if 'discount_category' not in test_df.columns:
        test_df['discount_category'] = global_stats['discount_category']['default']
    else:
        test_df['discount_category'] = (
            test_df['discount_category']
            .map(global_stats['discount_category'])
            .fillna(global_stats['discount_category']['default'])
        ).astype(int)

    # Manejar otras columnas faltantes o imputadas
    if 'similar_products_count' not in test_df.columns:
        test_df['similar_products_count'] = 0
    if 'cluster' not in test_df.columns:
        test_df['cluster'] = global_stats['cluster']['default']

    # Agregar columnas faltantes
    for feature in train_features:
        if feature not in test_df.columns:
            print(f"Agregando columna faltante: {feature}")
            test_df[feature] = 0

    # Convertir todas las columnas a tipos compatibles
    for col in test_df.columns:
        if col not in train_features + ['session_id']:
            continue
        if test_df[col].dtype == 'int64':
            test_df[col] = test_df[col].astype(np.int32)
        elif test_df[col].dtype == 'float64':
            test_df[col] = test_df[col].astype(np.float32)

    # Verificar que todas las columnas en `train_features` están presentes y tienen tipos válidos
    missing_features = [f for f in train_features if f not in test_df.columns]
    if missing_features:
        raise ValueError(f"Faltan características en el conjunto de prueba: {missing_features}")

    # Validar tipos finales
    invalid_dtypes = test_df.dtypes[~test_df.dtypes.isin([np.int32, np.float32, np.bool_])]
    if not invalid_dtypes.empty:
        raise ValueError(f"Columnas con tipos de datos no válidos: {invalid_dtypes}")

    return test_df[['session_id'] + train_features]

#### 3. **Entrenamiento con LambdaMART**
   - Utilizar las características generadas anteriormente para entrenar el modelo.
   - Excluir características no disponibles en `test` durante el entrenamiento.


In [5]:
def train_lambdamart_with_enrichment(train_path, model_path):
    print("\n--- Cargando datos de entrenamiento enriquecidos ---")
    train_df = pd.read_pickle(train_path)
    train_df = preprocess_and_generate_shared_features(train_df, mode="train")
    if "add_to_cart" not in train_df.columns:
        raise ValueError("La columna 'add_to_cart' no está presente en el conjunto de datos.")

    X = train_df.drop(['add_to_cart', 'session_id'], axis=1)
    y = train_df['add_to_cart']
    groups = train_df['session_id'].value_counts().values

    lgb_train = lgb.Dataset(X, label=y, group=groups)

    params = {
        'objective': 'lambdarank',
        'metric': 'ndcg',
        'ndcg_eval_at': [1, 3, 5],
        'learning_rate': 0.05,
        'num_leaves': 70,
        'max_bin': 255,
        'min_data_in_leaf': 20,
        'boosting_type': 'gbdt',
        'verbose': -1
    }

    model = lgb.train(
        params, lgb_train,
        num_boost_round=500,
        valid_sets=[lgb_train],
        valid_names=['train']
    )

    model.save_model(model_path)
    return X.columns.tolist()

#### 4. **Generación de Predicciones**
   - Utilizar el mismo conjunto de características para generar predicciones.
   - Aplicar el modelo entrenado al conjunto de prueba `test`.

In [6]:
def generate_predictions_with_enrichment(model_path, test_path, output_path, global_stats, train_features):
    print("\n--- Cargando el modelo entrenado ---")
    model = lgb.Booster(model_file=model_path)
    test_df = pd.read_pickle(test_path)
    test_df = preprocess_test_with_enrichment(test_df, global_stats, train_features)

    predictions = {}
    popular_products = test_df['partnumber'].value_counts().index.tolist()

    print("\n--- Generando predicciones para cada sesión ---")
    for session_id in test_df['session_id'].unique():
        session_data = test_df[test_df['session_id'] == session_id].copy()
        if session_data.empty:
            predictions[str(session_id)] = popular_products[:5]
            continue

        session_features = session_data[train_features]
        session_data['score'] = model.predict(session_features)

        recommended_products = (
            session_data.sort_values(by='score', ascending=False)['partnumber']
            .drop_duplicates()
            .tolist()
        )

        for product in popular_products:
            if len(recommended_products) >= 5:
                break
            if product not in recommended_products:
                recommended_products.append(product)

        predictions[str(session_id)] = recommended_products[:5]

    with open(output_path, 'w') as f:
        json.dump({"target": predictions}, f, indent=4)

---

### Pipeline

#### **1. Preprocesamiento y Generación de Características Compartidas**
- **Función: `preprocess_and_generate_shared_features`**
    - **Pros:**
        - Convierte fechas a numéricos.
        - Genera características clave como `product_popularity` y `session_length`.
        - Maneja columnas faltantes asignando valores predeterminados.
        - Elimina columnas con tipos de datos incompatibles.
    - **Preguntas clave:**
        - ¿Está `hour` siendo generada en algún paso previo en el `test`? Si no, necesitarías agregar ese cálculo aquí.
        - Si se espera agregar características personalizadas (como promedios de `discount_category` o `cluster`), ¿deberían incluirse aquí o en una etapa posterior?

---

#### **2. Generar Estadísticas Globales**
- **Función: `generate_global_stats`**
    - **Pros:**
        - Calcula estadísticas globales útiles como la popularidad de productos y la distribución de categorías.
        - Asigna valores predeterminados para características faltantes en el conjunto de prueba.
    - **Revisión:**
        - Las estadísticas generadas son adecuadas para imputar valores en el conjunto de prueba.
        - Asegúrate de que las claves y valores sean consistentes entre `train` y `test` (por ejemplo, `discount_category` podría no existir en `test`).

---

#### **3. Preprocesamiento del Conjunto de Prueba**
- **Función: `preprocess_test_with_enrichment`**
    - **Pros:**
        - Imputa características faltantes usando las estadísticas globales.
        - Genera características derivadas como `session_length` y `product_popularity`.
    - **Preguntas clave:**
        - ¿Es `similar_products_count` una característica relevante? Si no, podría omitirse para simplificar.
        - Validar que las columnas generadas coincidan exactamente con las del `train` después del preprocesamiento.

---

#### **4. Entrenamiento del Modelo**
- **Función: `train_lambdamart_with_enrichment`**
    - **Pros:**
        - Usa las características enriquecidas del conjunto de entrenamiento.
        - Configura el modelo Lambdamart con parámetros estándar de `LightGBM`.
    - **Preguntas clave:**
        - ¿Se ha probado la robustez del modelo al eliminar características no compartidas del `train`? Esto asegura que el modelo no dependa de datos que no estarán disponibles en el `test`.

---

#### **5. Generación de Predicciones**
- **Función: `generate_predictions_with_enrichment`**
    - **Pros:**
        - Garantiza que cada `session_id` tenga exactamente 5 productos únicos.
        - Usa características derivadas e imputadas en el `test`.
    - **Revisión:**
        - Validar que los productos recomendados no incluyan duplicados y estén ordenados por puntuación predicha.
        - Asegurar que las características usadas para predecir sean exactamente las mismas que durante el entrenamiento.

---

### **Resumen**

El pipeline es sólido y completo, pero aquí hay algunos puntos clave a revisar:
1. **Validación de Columnas Compartidas:**
    - Garantizar que las columnas generadas en `test` coincidan exactamente con las del `train` después del preprocesamiento.
    - Eliminar cualquier columna irrelevante o no compatible antes de alimentar el modelo.

2. **Manejo de Datos Faltantes:**
    - Las estadísticas globales son esenciales para rellenar datos faltantes en `test`.
    - Validar que las imputaciones no introduzcan sesgos significativos.

3. **Predicciones:**
    - Garantizar que las predicciones generen 5 productos únicos por sesión.
    - Asegurar que los productos más populares sean usados como respaldo para sesiones con datos insuficientes.


---

#### **1. Generar las Estadísticas Globales**
Ejecuta el siguiente código para calcular las estadísticas globales basadas en el conjunto de entrenamiento enriquecido:



In [ ]:
train_path = '/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/train_data_final.pkl'

# Cargar datos de entrenamiento
train_df = pd.read_pickle(train_path)

# Generar estadísticas globales
global_stats = generate_global_stats(train_df)
# print(global_stats)


#### **2. Entrenar el Modelo**
Entrena el modelo Lambdamart usando las características enriquecidas del conjunto de entrenamiento:



In [ ]:
model_path = '/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/models/lambdamart_enriched_shared_model.txt'

# Entrenar modelo y obtener características de entrenamiento
train_features = train_lambdamart_with_enrichment(train_path, model_path)


In [ ]:
# print(train_features)

#### **3. Generar las Predicciones**
Usa el modelo entrenado para generar predicciones basadas en el conjunto de prueba y las estadísticas globales:



In [ ]:
test_path = '/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/test_data.pkl'
output_path = '/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/predictions/predictions_3_shared.json'

# Generar predicciones
generate_predictions_with_enrichment(model_path, test_path, output_path, global_stats, train_features)


#### **4. Validar el JSON**
Valida que el JSON generado cumpla con el formato requerido:


In [ ]:
import json

# Validar formato del JSON
with open(output_path, 'r') as f:
    predictions = json.load(f)

# Mostrar una parte del JSON para verificación
print("\n--- Validación del JSON generado ---")
print(json.dumps(predictions, indent=4)[:1000])  # Mostrar los primeros 1000 caracteres
